<a href="https://colab.research.google.com/github/ChavChavC/BT4222/blob/main/Feature_Engineering1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader

dataset = load_dataset("Jean-Baptiste/financial_news_sentiment")

test

This is an example text.

Models to do:

1.   DistilBERT
2.   BERT
3.   FinBERT
4.   SVM
5.   RF
6.   Roberta-fin (their model) lmao

ROBERTA (their model)

In [13]:
# ROBERTA FIN BERT model
#Their own model
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
model = AutoModelForSequenceClassification.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-financial-news-sentiment-en")

classifier=pipeline("text-classification",model=model, tokenizer=tokenizer)
output=classifier(dataset['train']['summary_detail_with_title'][0])
print(output)

[{'label': 'positive', 'score': 0.9405916333198547}, {'label': 'neutral', 'score': 0.9450716376304626}, {'label': 'positive', 'score': 0.9427127838134766}, {'label': 'positive', 'score': 0.9426810145378113}, {'label': 'positive', 'score': 0.9421485662460327}, {'label': 'neutral', 'score': 0.9456402063369751}, {'label': 'positive', 'score': 0.9421244263648987}, {'label': 'neutral', 'score': 0.9445323348045349}, {'label': 'neutral', 'score': 0.9450904726982117}, {'label': 'positive', 'score': 0.9429874420166016}, {'label': 'neutral', 'score': 0.9449349641799927}, {'label': 'neutral', 'score': 0.945558488368988}, {'label': 'positive', 'score': 0.9414768815040588}, {'label': 'neutral', 'score': 0.9459429383277893}, {'label': 'positive', 'score': 0.9428868889808655}, {'label': 'positive', 'score': 0.9431929588317871}, {'label': 'neutral', 'score': 0.9440200924873352}, {'label': 'neutral', 'score': 0.9446520805358887}, {'label': 'neutral', 'score': 0.94461989402771}, {'label': 'positive', 's

DistilBERT / assuming BERT will be similar just that we changing input of 'from_pretrained()' (?)

In [ ]:
# Distilbert
from transformers import AutoTokenizer, DistilBertTokenizer, DistilBertModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer,DataCollatorWithPadding
import numpy as np
import evaluate

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["summary_detail_with_title"], truncation=True)
tokenizeddataset = dataset.map(preprocess_function, batched=True)


id2label = {0: "NEGATIVE", 1: "NEUTRAL", 2:"POSITIVE"}
label2id = {"NEGATIVE": 0, "NEUTRAL":1 , "POSITIVE": 2}


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir= "my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizeddataset["train"],
    eval_dataset=tokenizeddataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
trainer.train()


In [ ]:
classifier=pipeline("text-classification",model=model, tokenizer=tokenizer)
output=classifier(dataset['train']['summary_detail_with_title'][0])
print(output)